<a href="https://colab.research.google.com/github/ccarpenterg/introNLP/blob/master/03a_NLP_and_recurrent_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP and Recurrent Neural Networks

In [0]:
# load Tensorflow 2
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np

print(tf.__version__)

## Sentiment Analysis of IMDB Movie Reviews

In [0]:
train_validation_split = tfds.Split.TRAIN.subsplit([8, 2])

(train_validation_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (train_validation_split, tfds.Split.TEST),
    as_supervised=True,
    with_info=True
)

train_data, validation_data = train_validation_data

In [0]:
encoder = info.features['text'].encoder

## Preprocessing the Dataset with Padding

In [0]:
BUFFER_SIZE = 5000
BATCH_SIZE = 32

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_data))
)

validation_batches = (
    validation_data
    .padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(validation_data))
)

test_batches = (
    test_data
    .padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_data))
)

In [0]:
for batch_example, labels in train_batches.take(2):
    print("Batch shape:", batch_example.shape)
    print("Labels shape:", labels.shape)

## Sequence Processing with a Recurrent Neural Network

In [0]:
model_v1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_v1.summary()

In [0]:
model_v2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_v2.summary()

**Embeding layer**



**LSTM layer**

Cell: 64 x 64 + 64 x 64 (weights) + 64 (biases) = 8,256 parameters

Update gate: 64 x 64 + 64 x 64 (weights) + 64 (biases) = 8,256 parameters

Forget gate: 64 x 64 + 64 x 64 (weights) + 64 (biases) = 8,256 parameters

Output gate: 64 x 64 + 64 x 64 (weights) + 64 (biases) = 8,256 parameters

**Bidirectional**

Forward LSTM: 33,024 parameters

Backward LSTM: 33,024 parameters

Total: 66,048 parameters

**FC layer**

The bidirectional layer outputs two activationn vectors, one for the forward LSTM and one for the backward LTSM, that gives us 128 activation units.

64 * 128 weigths + 64 biases = 8,256 parameters

**Dense classifier layer**

1 * 64 weights + 1 bias = 64 parameters

In [0]:
model_v3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_v3.summary()

## Training Our RNNs

In [0]:
EPOCHS = 10
VAL_STEPS = 30
results = {}

In [0]:
model_v1.compile(loss='binary_crossentropy',
                 optimizer=tf.keras.optimizers.Adam(1e-4),
                 metrics=['accuracy'])

results['v1'] = model_v1.fit(train_batches, epochs=EPOCHS,
                             validation_data=validation_batches,
                             validation_steps=VAL_STEPS)

In [0]:
test_loss, test_accuracy = model_v1.evaluate(test_batches, verbose=0)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_accuracy))

In [0]:
model_v2.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

results['v2'] = model_v2.fit(train_batches, epochs=EPOCHS,
                             validation_data=validation_batches,
                             validation_steps=VAL_STEPS)

In [0]:
test_loss, test_accuracy = model_v2.evaluate(test_batches, verbose=0)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_accuracy))

In [0]:
model_v3.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

results['v3'] = model_v3.fit(train_batches, epochs=EPOCHS,
                             validation_data=validation_batches,
                             validation_steps=VAL_STEPS)

In [0]:
test_loss, test_accuracy = model_v3.evaluate(test_batches, verbose=0)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_accuracy))

In [0]:
import matplotlib.pyplot as plt

def plot_stats(training_results):

    training_dict = training_results.history

    acc = training_dict['accuracy']
    val_acc = training_dict['val_accuracy']
    loss = training_dict['loss']
    val_loss = training_dict['val_loss']

    epochs = range(1, EPOCHS + 1)

    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.figure()

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [0]:
plot_stats(results['v1'])

In [0]:
plot_stats(results['v2'])

In [0]:
plot_stats(results['v3'])